In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

导入相关库

In [ ]:
import  torch.utils.data as data
import os
import torch
import numpy as np
import glob
import pandas as pd
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
from tqdm import  tqdm
from torchsummary import summary
from torch.autograd import Variable

制作数据载入器

In [ ]:
root = "/workspace/Data/modelnet40_normal_resampled"
npoints = 9999
split = "train"
data_augmentation = True

In [ ]:
cls_names = glob.glob(os.path.join(root,"*/*.txt"))
cls_names = [p.split("/")[4] for p in cls_names]
cls_names = np.unique(cls_names)

In [ ]:
clsname_to_index = dict((name,index) for index,name in enumerate(cls_names))

In [ ]:
class ModelNetDataset(data.Dataset):
  def __init__(self,
               root,
               classmap,
               npoints = 9999,
               split = "train",
               data_augmentation = True
               ):
    self.classmap = classmap
    self.npoints = npoints
    self.root = root
    self.split = split
    self.data_augmentation = data_augmentation
    self.point_paths = []
    self.labels = []

    list_file = None
    if split == "train":
      list_file = os.path.join(root,"modelnet40_train.txt")
    else:
      list_file = os.path.join(root,"modelnet40_test.txt")

    # 提取对应的文件列表
    files = []
    if list_file != None:
      with open(list_file, 'r',encoding="utf-8") as f:
        for line in f.readlines():
          file_name = line.replace("\n","")
          files.append(file_name) 
    else:
      print("ERROR:list file not exist!!!")

    paths = glob.glob(os.path.join(root,"*/*.txt"))
    self.point_paths = [p for p in paths if p.split("/")[5].replace(".txt","") in files]
    self.labels = [self.classmap.get(p.split("/")[4]) for p in self.point_paths]

  def __getitem__(self, index):

    point_path = self.point_paths[index]
    label = self.labels[index]
    points = pd.read_csv(point_path).iloc[:,0:3]
    points = np.asarray(points)


    # 归一化
    # points = points - np.expand_dims(np.mean(points,axis=0),0)
    points = points - np.mean(points,axis=0)
    dist = np.max(np.linalg.norm(points,axis=1))
    points = points/dist

    # 随机旋转角度和添加噪声
    if (self.split == "train" and self.data_augmentation):
      theta = np.random.uniform(0, np.pi * 2)
      rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
      points[:, [0, 2]] = rotation_matrix.dot(points[:, [0, 2]].T).T              # random rotation
      points += np.random.normal(0, 0.02, (points.shape))                         # random jitter

    np.random.shuffle(points)

    points = torch.from_numpy(points.astype(np.float32))
    label = torch.from_numpy(np.array([label]).astype(np.int64))

    return points,label

  def __len__(self):
    return len(self.point_paths)



In [ ]:
batch_size = 16

In [ ]:
traindatasets = ModelNetDataset(root,clsname_to_index,split="train",data_augmentation=True)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(traindatasets,
                                               batch_size = batch_size,
                                               shuffle = True)

In [ ]:
testdatasets = ModelNetDataset(root,clsname_to_index,split = "test",data_augmentation=False)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(testdatasets,
                                               batch_size = batch_size,
                                               shuffle = True)

定义网络

In [ ]:
class STN3d(nn.Module): # 未保证旋转矩阵的特性
  def __init__(self):
    super(STN3d, self).__init__()
    self.conv1 = torch.nn.Conv1d(3, 64, 1)
    self.conv2 = torch.nn.Conv1d(64, 128, 1)
    self.conv3 = torch.nn.Conv1d(128, 1024, 1)
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 9)
    self.relu = nn.ReLU()

    self.bn1 = nn.BatchNorm1d(64)
    self.bn2 = nn.BatchNorm1d(128)
    self.bn3 = nn.BatchNorm1d(1024)
    self.bn4 = nn.BatchNorm1d(512)
    self.bn5 = nn.BatchNorm1d(256)


  def forward(self, x):
    batchsize = x.size()[0]
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = nn.MaxPool1d(x.size(-1))(x)
    x = nn.Flatten(1)(x)

    x = F.relu(self.bn4(self.fc1(x)))
    x = F.relu(self.bn5(self.fc2(x)))
    x = self.fc3(x)

    iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
    if x.is_cuda:
      iden = iden.cuda()
    x = x + iden
    x = x.view(-1, 3, 3)
    return x

In [ ]:
class PointNetfeat(nn.Module):
  def __init__(self):
    super(PointNetfeat, self).__init__()
    self.stn = STN3d()
    self.conv1 = torch.nn.Conv1d(3, 64, 1)
    self.conv2 = torch.nn.Conv1d(64, 128, 1)
    self.conv3 = torch.nn.Conv1d(128, 1024, 1)
    self.bn1 = nn.BatchNorm1d(64)
    self.bn2 = nn.BatchNorm1d(128)
    self.bn3 = nn.BatchNorm1d(1024)

  def forward(self, x):
    # trans = self.stn(x)
    # x = x.transpose(2, 1)
    # x = torch.bmm(x, trans)
    # x = x.transpose(2, 1)
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = self.bn3(self.conv3(x))
    x = nn.MaxPool1d(x.size(-1))(x)
    x = nn.Flatten(1)(x)
      
    return x

In [ ]:
class PointNetCls(nn.Module):
  def __init__(self, k=40):
    super(PointNetCls, self).__init__()
    self.feat = PointNetfeat()
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, k)
    self.dropout = nn.Dropout(p=0.3)
    self.bn1 = nn.BatchNorm1d(512)
    self.bn2 = nn.BatchNorm1d(256)

  def forward(self, x):
    x = self.feat(x)
    x = F.relu(self.bn1(self.fc1(x)))
    x = F.relu(self.bn2(self.dropout(self.fc2(x))))
    x = self.fc3(x)
    return F.log_softmax(x, dim=1)

In [ ]:
classifier = PointNetCls(k=40)

In [ ]:
classifier.cuda()

In [ ]:
summary(classifier,(3,9999))

In [ ]:
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

In [ ]:
num_batch = len(traindatasets) / batch_size

In [ ]:
blue = lambda x: '\033[94m' + x + '\033[0m'

In [ ]:
for epoch in range(15):
  classifier = classifier.train()
  running_loss = 0.0
  for i, data in enumerate(train_dataloader, 0):

    points, target = data
    target = target[:, 0]
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()

    optimizer.zero_grad()
    pred = classifier(points)

    loss = F.nll_loss(pred, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if i % 10 == 9:
      print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                        (epoch + 1, i + 1, len(train_dataloader), running_loss / 10))
      running_loss = 0.0
  
  classifier = classifier.eval()
  correct = total = 0

  with torch.no_grad():
    for data in test_dataloader:
      points, target = data
      target = target[:, 0]
      points = points.transpose(2, 1)
      points, target = points.cuda(), target.cuda()
      pred = classifier(points)
      loss = F.nll_loss(pred, target)
      pred_choice = pred.data.max(1)[1]
      correct += pred_choice.eq(target.data).cpu().sum()
      total += target.data.size(0)

  val_acc = 100. * correct / total
  print('Valid accuracy: %d %%' % val_acc)


In [ ]:
total_correct = 0
total_testset = 0
for i,data in tqdm(enumerate(test_dataloader, 0)):
  points, target = data
  target = target[:, 0]
  points = points.transpose(2, 1)
  points, target = points.cuda(), target.cuda()
  classifier = classifier.eval()
  pred = classifier(points)
  pred_choice = pred.data.max(1)[1]
  correct = pred_choice.eq(target.data).cpu().sum()
  total_correct += correct.item()
  total_testset += points.size()[0]


In [ ]:
print("final accuracy {}".format(total_correct / float(total_testset)))